### <div align="center">Introduction</div>
Even after the 2022/2023 soccer season was over, I had a burning desire to do a pet project and analyze the data from the current season. For a long time I delayed this moment and could not even imagine that by the beginning of August the data for the last season would simply not be freely available on the Internet. Therefore, I decided to work with Internet data on my own, in particular, to learn how to use web scraping and collect data for scientific and research purposes

This part of the project will be devoted to web scraping and data collection from __[understat.com](https://understat.com)__. I initially relied on this author's __[notebook](https://www.kaggle.com/code/slehkyi/web-scraping-football-statistics-per-game-data/notebook)__, for which I thank him very much, and also supplemented it with other data from site. 

Of course, there are many interactive graphs and tables on these kinds of sites, but sometimes you want to look and build something of your own. In particular, as an Atletico Madrid fan, I would like to see how successful this season has been for a part of players and the team as whole.


### Loading of necessary libraries for work with data:

In [1]:
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import json
import plotly.express as px

### Some information about data

#### Let's repeat about the data structure of understat.com. 
On the main page of the site the user can choose one of 6 leagues(I will take only the top 5 leagues without the RFPL):
* EPL 
* La liga
* Ligue 1
* Serie A
* Bundesliga
* RFPL

By clicking on each tab, we can select any season we are interested in from 2014/2015 through 2023/2024. In addition, we can also go to the team page and detailed stats for each player of the team

The author's notebook described the process of scraping these leagues. However, this didn't seem enough to me and I wanted to get player and team data as well

The foundation of the basics in this approach is the URL. The basic structure is clear - https://understat.com. Depending on the information we want to retrieve, the additional URL changes:
* for <span style="color:red">league</span> it is <b>league/</b> + <b>league_name</b> + <b>/season</b>
* for a <span style="color:red">team</span> it is <b>team/</b> + <b>team_name</b> + <b>/season</b>
* most difficult for the <span style="color:red">player</span>. Here we build everything by player id. That is <b>player/</b> + <b>player_id</b>

By analogy, I found the location of the data I needed. By tagging "script" with BeautifulSoup, I will find the necessary parts for each URL:
* for <span style="color:red">leagues</span> it is <b>'teamsData'</b> with information and advanced statistics for each played match of a team.
* for <span style="color:red">teams</span> it is <b>'playersData'</b> with brief statistics of each player of the team in the season
* for a <span style="color:red">player</span> there were several data, but I decided to take 2: <b>'shotsData'</b> with information on each shot of a certain player and <b>'matchesData'</b> with information on the match in which the player participated.

I decided not to write a separate function for each item and made everything in one function:

In [2]:
def get_understat_data(item_type, # can be 3 types: team, player and league
                       identifier = None, # is specified only for item_type == league or team
                       season = None, # is specified only for item_type == league or team
                       player_data_type = None): # is specified only for item_type == player. Can be 'shots' or 'matches'
    
    base_url = 'https://understat.com/'
    
    #URL and data_key generation depending on the item_type
    if item_type == 'team':
        data_key = 'playersData'
        url = f'{base_url}team/{identifier}/{season}' if identifier and season else None
        
    elif item_type == 'player':
        url = f'{base_url}player/{identifier}' if identifier else None
        
        if player_data_type == 'shots':
            data_key = 'shotsData'   
        elif player_data_type == 'matches':
            data_key = 'matchesData'
        else:
            print(f'Invalid player data type "{player_data_type}". Valid types are "shots" or "matches"')
            return None
        
    elif item_type == 'league':
        data_key = 'teamsData'
        url = f'{base_url}league/{identifier}/{season}' if identifier and season else None
    else:
        print(f'Invalid item type: "{item_type}". Valid types are "player", "team", "league".')
        return None
    
    if not url:
        print(f'Invalid combination of identifier and season for item type "{item_type}".')
        return None
    
    try:
        # Making a get request
        res = requests.get(url)
        
        # Check if an error has occured
        res.raise_for_status()
        
        # Use lxml parser
        soup = BeautifulSoup(res.content, "lxml")
        
        # Find strings with 'script'
        scripts = soup.find_all('script')
        
        # Find the required data by data_key and strip it 
        string_with_json_obj = next(el.text.strip() for el in scripts if data_key in el.text)
        ind_start = string_with_json_obj.index("('") + 2
        ind_end = string_with_json_obj.index("')")
        json_data = string_with_json_obj[ind_start:ind_end]
        
        # Decode the JSON data, converting any Unicode sequences into corresponding Unicode characters
        json_data = json_data.encode('utf8').decode('unicode_escape')
        
        # Load data in Python dict
        data = json.loads(json_data)
        
        return data
    
    except Exception as e:
        print(f"Error occurred while processing {item_type}: {e}")
        return None

### Work with leagues

One of the most difficult moments. The code turned out to be very cumbersome because of nested dictionary and with a large number of iterations. There might have been a way to make it simpler and faster, but I decided to make the code easier to understand

In [26]:
# Lists for iterations
leagues = ['EPL', 'La liga', 'Bundesliga', 'Serie A', 'Ligue 1']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
# We will need this to autocomplete the team names
team_titles_dict = {}

for league in leagues:
    for season in seasons:
        # Call the function for league
        data = get_understat_data(item_type = 'league', identifier = league, season = season)
        # Create a dataframe for adding individual dataframes in the future
        season_league_df = pd.DataFrame()
        
        for league_season_data in [data]:
            # Because of nested dictionary peculiarity, 
            # we have to iterate through the next iteration to get to the innards
            for team_id, team_data in league_season_data.items():
                id_value = team_data['id']
                title_value = team_data['title']
                
                #Making a dict of the form {season : [team_names]}
                if season in team_titles_dict:
                    team_titles_dict[season].append(title_value)
                else:
                    team_titles_dict[season] = [title_value]
               
                #The very important innards that we'll be unpacking
                for history_item in team_data['history']:
                    
                    unpacked_item = {'id' : id_value, 'title' : title_value}
                    # Add 'id' and 'team_name' for every row
                    unpacked_item.update(history_item)
                    
                    # Unpack ppda
                    ppda_dict = unpacked_item.pop('ppda', {})
                    unpacked_item['att_ppda'] = ppda_dict.get('att')
                    unpacked_item['def_ppda'] = ppda_dict.get('def')
                    
                    # Unpack ppda_allowed
                    ppda_dict = unpacked_item.pop('ppda_allowed', {})
                    unpacked_item['att_ppda_allowed'] = ppda_dict.get('att')
                    unpacked_item['def_ppda_allowed'] = ppda_dict.get('def')
                    
                    # Obtaining the final dataframe for a particular team in a particular year
                    season_league_df = season_league_df.append((pd.DataFrame(unpacked_item.values(), unpacked_item.keys())).transpose())
        
        '''Uncomment to load each dataframe in csv format ↓ '''
        #season_league_df.to_csv(f'{league}_{season}_data.csv', index = False)
                


In [25]:
# Let's look at the last iterated dataframe and the columns in it
display(season_league_df.sample(4), season_league_df.columns)

,id,title,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,...,date,wins,draws,loses,pts,npxGD,att_ppda,def_ppda,att_ppda_allowed,def_ppda_allowed
0,164,Marseille,a,1.30958,1.64132,1.30958,1.64132,3,5,1,...,2022-08-14 18:45:00,0,1,0,1,-0.33174,197,20,299,27
0,210,Lens,h,2.29667,0.18809,1.53657,0.18809,14,1,3,...,2023-05-27 19:00:00,1,0,0,3,1.34848,93,19,255,8
0,174,Toulouse,a,2.52738,0.211301,2.52738,0.211301,6,6,0,...,2023-05-07 13:00:00,0,1,0,1,2.316079,195,22,298,16
0,210,Lens,h,1.31665,0.580047,1.31665,0.580047,9,9,2,...,2023-04-07 19:00:00,1,0,0,3,0.736603,218,20,325,38


Index(['id', 'title', 'h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'deep',
       'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins',
       'draws', 'loses', 'pts', 'npxGD', 'att_ppda', 'def_ppda',
       'att_ppda_allowed', 'def_ppda_allowed'],
      dtype='object')

### Work with teams

It's simpler than that. Iterating through the leagues helped us create a dict of team names. 

I decided to make a single file with data for each season instead of splitting the data for each league and year again

In [27]:
# A list of dataframes, and that says it all.
dataframes_list = []

for season, teams in team_titles_dict.items():
    for team in teams:
        # Call the function for team
        data = get_understat_data(item_type = 'team', identifier=team, season=season)
        current_df = pd.DataFrame(data)
        # Add for the current_df season date
        current_df['season'] = season
        dataframes_list.append(current_df)
        
#Combine all dataframes to get a total with statistics of players in the team
teams_data = pd.concat(dataframes_list, ignore_index = True)

In [34]:
# Let's look at some rows for the dataframe
teams_data.sample(4)

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup,season
10641,3306,Remi Walter,12,1000,0,0.342293594032526,1,0.09275249671190977,7,2,1,0,M,Troyes,0,0.342293594032526,2.1032978324219584,1.6825675023719668,2017
8430,6249,Davinson Sánchez,31,2535,0,0.5712869055569172,0,0.12518769968301058,9,4,1,1,D S,Tottenham,0,0.5712869055569172,7.701591860502958,7.640092059969902,2017
7754,3212,Zinedine Machach,10,419,0,0.12628620024770498,0,0.29655829817056656,6,4,3,1,M S,Marseille,0,0.12628620024770498,1.0404028128832579,0.7299971077591181,2016
952,2509,Dimitri Foulquier,25,1683,0,0.7044909531250596,2,1.9348381673917174,7,13,6,0,D F M S,Granada,0,0.7044909531250596,3.61661705840379,1.28240398876369,2014


In [35]:
# Save teams dataframe to csv
#teams_data.to_csv('team_players_stat.csv', index = False)

### Work with players

Of course, similar to the team names, we could create a list with interesting for us player ids for the top 5 leagues in the team work, but I decided to upload all players' data (the maximum ids that were found was 11689. It is possible that there is more data already)

In [35]:
# It was interesting how long it would take to complete such an extensive job
import time
start_time = time.time()
# A list of dataframes for matches and shots
dataframes_list_for_shots = []
dataframes_list_for_matches = []

for identifier in range(1,11689+1):
    
    #  Call the function for player and data_type == 'shots'
    data_1 = get_understat_data(item_type='player', identifier=identifier, player_data_type = 'shots')
    current_df_1 = pd.DataFrame(data_1)
    dataframes_list_for_shots.append(current_df_1)
    
    # Call the function for player and data_type == 'matches'
    data_2 = get_understat_data(item_type='player', identifier=identifier, player_data_type = 'matches')
    current_df_2 = pd.DataFrame(data_2)
    dataframes_list_for_matches.append(current_df_2)
    

# Final dataframes
player_shots_data = pd.concat(dataframes_list_for_shots, ignore_index = True)
player_matches_data = pd.concat(dataframes_list_for_matches, ignore_index = True)
print("--- %s seconds ---" % (time.time() - start_time))

Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5835
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5835
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5836
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5836
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5837
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5837
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5838
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5838
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5839
E

Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5872
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5873
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5873
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5874
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5874
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5875
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5875
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5876
Error occurred while processing player: 404 Client Error: Not Found for url: https://understat.com/player/5876
E

A lot of 404 errors with missing player data popped up, but it didn't affect the code execution in any way 

In [66]:
# Let's look at the players_shots dataframe and the columns in it
display(player_shots_data.sample(4), player_shots_data.columns)

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
50232,399110,50,BlockedShot,0.955,0.43099998474121093,0.14286300539970398,Olivier Giroud,a,502,OpenPlay,2020,RightFoot,14617,Fulham,Chelsea,0,1,2021-01-16 17:30:00,Mason Mount,Pass
161322,121386,51,MissedShots,0.905,0.6780000305175782,0.09102659672498703,Marcos Alonso,h,1621,OpenPlay,2016,LeftFoot,3493,Chelsea,Middlesbrough,3,0,2017-05-08 20:00:00,Pedro,Pass
252906,262980,11,Goal,0.855,0.43099998474121093,0.12444400042295456,Joãozinho,h,2928,OpenPlay,2018,LeftFoot,9078,Dinamo Moscow,Ural,4,0,2018-12-09 11:00:00,Evgeni Lutsenko,Pass
30493,134846,22,MissedShots,0.7559999847412109,0.655,0.022783899679780006,Marcel Schmelzer,h,313,OpenPlay,2016,LeftFoot,3860,Borussia Dortmund,Hoffenheim,2,1,2017-05-06 14:30:00,None,None


Index(['id', 'minute', 'result', 'X', 'Y', 'xG', 'player', 'h_a', 'player_id',
       'situation', 'season', 'shotType', 'match_id', 'h_team', 'a_team',
       'h_goals', 'a_goals', 'date', 'player_assisted', 'lastAction'],
      dtype='object')

In [44]:
# Save it to csv
#player_shots_data.to_csv('shots_statistics.csv', index=False)

In [67]:
# Let's look at the players_matches dataframe and the columns in it
display(player_matches_data.sample(4), player_shots_data.columns)

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,id,season,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup
201925,0,1,0.04097370058298111,36,AMC,Hamburger SV,Paderborn,0,3,2014-08-30,5459,2014,27927,0,0,0,0,0.04097370058298111,0,0
307521,0,2,0.08531410247087479,90,FWR,Wolverhampton Wanderers,Watford,0,2,2018-10-20,9278,2018,276293,0.014257100410759449,0,1,0,0.08531410247087479,0.09957130253314972,0.014257100410759449
189606,0,1,0.04123780131340027,90,MC,Verona,Empoli,2,1,2015-05-17,4804,2014,55413,0.04969099909067154,0,1,0,0.04123780131340027,0.36219900846481323,0.3125079870223999
343293,0,1,0.07298289984464645,24,Sub,Union Berlin,Mainz 05,1,1,2020-05-27,12651,2019,398821,0,0,0,0,0.07298289984464645,0.07298289984464645,0


Index(['id', 'minute', 'result', 'X', 'Y', 'xG', 'player', 'h_a', 'player_id',
       'situation', 'season', 'shotType', 'match_id', 'h_team', 'a_team',
       'h_goals', 'a_goals', 'date', 'player_assisted', 'lastAction'],
      dtype='object')

In [68]:
# Save it to csv
# player_matches_data.to_csv('matches_statistics.csv', index=False)